In [6]:
%load_ext autoreload
%autoreload 2

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import datetime

In [124]:
def get_all_data():
    # Parse data.json file
    data = requests.get('https://api.covid19india.org/data.json').json()

    # Create data frame for testing data
    df_tested = pd.DataFrame.from_dict(data['tested'])
    df_tested = df_tested[np.logical_not(df_tested['source'] == '')]
    df_tested.reset_index(inplace=True, drop=True)

    # Create data frame for statewise data
    for i, x in enumerate(data['statewise']):
        delta_dict = dict([('delta_'+k, v) for k, v in x['delta'].items()])
        data['statewise'][i].update(delta_dict)
        del data['statewise'][i]['delta']
    df_statewise = pd.DataFrame.from_dict(data['statewise'])

    # Create data frame for time series data
    df_india_time_series = pd.DataFrame.from_dict(data['cases_time_series'])
    df_india_time_series['date'] = pd.to_datetime([datetime.datetime.strptime(x[:6]+ ' 2020', '%d %b %Y') for x in df_india_time_series['date']])
    
    # Parse state_district_wise.json file
    data = requests.get('https://api.covid19india.org/state_district_wise.json').json()
    states = data.keys()
    for state in states:
        for district, district_dict in data[state]['districtData'].items():
            delta_dict = dict([('delta_'+k, v) for k, v in district_dict['delta'].items()])
            data[state]['districtData'][district].update(delta_dict)
            del data[state]['districtData'][district]['delta']

    columns = ['state', 'district', 'confirmed', 'delta_confirmed', 'lastupdatedtime']
    df_districtwise = pd.DataFrame(columns=columns)
    for state in states:
        df = pd.DataFrame.from_dict(data[state]['districtData']).T.reset_index()
        df.columns = ['district', 'confirmed', 'delta_confirmed', 'lastupdatedtime']
        df['state'] = state
        df = df[columns]
        df_districtwise = pd.concat([df_districtwise, df], ignore_index=True)
        
    # Parse raw_data.json file
    # Create dataframe for raw history
    data = requests.get('https://api.covid19india.org/raw_data.json').json()
    df_raw_data = pd.DataFrame.from_dict(data['raw_data'])
    df_raw_data = df_raw_data[np.logical_not(df_raw_data['source1'] == '')]

    # Parse travel_history.json file
    # Create dataframe for travel history
    data = requests.get('https://api.covid19india.org/travel_history.json').json()
    df_travel_history = pd.DataFrame.from_dict(data['travel_history'])
    
    return df_tested, df_statewise, df_india_time_series, df_districtwise, df_raw_data, df_travel_history

In [125]:
df_tested, df_statewise, df_india_time_series, df_districtwise, df_raw_data, df_travel_history = get_all_data()

In [131]:
df_travel_history

,_cn6ca,_d2mkx,accuracylocation,address,datasource,latlong,modeoftravel,pid,placename,timefrom,timeto,type
0,1,NaN,,"Hotel Aryas, Kuthattukulam, Muvattupuzha road",https://drive.google.com/drive/folders/1yAgt3I...,"9.8614231,76.5900643",,P35,,29/02/2020 10:30:00,29/02/2020 11:30:00,placeVisit
1,2,NaN,,Suresh Hotel Ranni,https://drive.google.com/drive/folders/1yAgt3I...,"9.3823686,76.779199",,P35,,01/03/2020 21:30:00,01/03/2020 23:00:00,placeVisit
2,3,NaN,,"Post Office Ranni Pazvangadi, Ranni",https://drive.google.com/drive/folders/1yAgt3I...,"9.3757533,76.7763825",,P35,,02/03/2020 11:00:00,02/03/2020 11:30:00,placeVisit
3,4,NaN,,"Knanaya Church, Pazvangadi Ranni",https://drive.google.com/drive/folders/1yAgt3I...,"9.375753,76.7698164",,P35,,02/03/2020 11:30:00,02/03/2020 12:00:00,placeVisit
4,5,NaN,,Post Office Ranni Pazvangadi,https://drive.google.com/drive/folders/1yAgt3I...,"9.3757533,76.7763825",,P35,,02/03/2020 12:00:00,02/03/2020 13:00:00,placeVisit
5,6,NaN,,"Golde Emporium, New Hypermarket",https://drive.google.com/drive/folders/1yAgt3I...,"9.385127,76.7849146",,P35,,02/03/2020 13:15:00,02/03/2020 14:00:00,placeVisit
6,7,NaN,,"Muthoot mini supermarket, Ranni",https://drive.google.com/drive/folders/1yAgt3I...,,,P35,,02/03/2020 14:30:00,,placeVisit
7,8,NaN,,"Imperial Backers, Punalur",https://drive.google.com/drive/folders/1yAgt3I...,"9.0164797,76.92397",,P35,,02/03/2020 18:00:00,,placeVisit
8,9,NaN,,"Relative house, Manjar PO, Punalur",https://drive.google.com/drive/folders/1yAgt3I...,"9.0158521,76.9097683",,P35,,02/03/2020 19:00:00,,placeVisit
9,10,NaN,,"SBI/SBT Ranni, Thottamon",https://drive.google.com/drive/folders/1yAgt3I...,"9.3772508,76.7756069",,P35,,03/03/2020 12:00:00,,placeVisit
